<a href="https://colab.research.google.com/github/tenoob/mushroom-classification/blob/main/training_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install prince

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.8/415.8 kB 5.3 MB/s eta 0:00:00


In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("/content/drive/MyDrive/artificial intelligence/preprocessed_dataset.csv")

In [5]:
df.head()

,Unnamed: 0,target,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,0,p,5,2,4,1,6,1,0,1,...,2,7,7,0,2,1,4,2,3,5
1,1,e,5,2,9,1,0,1,0,0,...,2,7,7,0,2,1,4,3,2,1
2,2,e,0,2,8,1,3,1,0,0,...,2,7,7,0,2,1,4,3,2,3
3,3,p,5,3,8,1,6,1,0,1,...,2,7,7,0,2,1,4,2,3,5
4,4,e,5,2,3,0,5,1,1,0,...,2,7,7,0,2,1,0,3,0,1


In [6]:
x = df.iloc[:,2:]
x.head()

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,5,2,4,1,6,1,0,1,4,0,...,2,7,7,0,2,1,4,2,3,5
1,5,2,9,1,0,1,0,0,4,0,...,2,7,7,0,2,1,4,3,2,1
2,0,2,8,1,3,1,0,0,5,0,...,2,7,7,0,2,1,4,3,2,3
3,5,3,8,1,6,1,0,1,5,0,...,2,7,7,0,2,1,4,2,3,5
4,5,2,3,0,5,1,1,0,4,1,...,2,7,7,0,2,1,0,3,0,1


In [7]:
#dimentionality reduction
"""import prince
mca = prince.MCA()
x_mca = mca.fit_transform(x)
x_mca.head()"""

'import prince\nmca = prince.MCA()\nx_mca = mca.fit_transform(x)\nx_mca.head()'

In [8]:
y = df.iloc[:,1]
y.head()

,target
0,p
1,e
2,e
3,p
4,e


In [9]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

In [10]:
y

array([1, 0, 0, ..., 0, 1, 0])

In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=45)

In [12]:
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(6499, 22) (6499,)
(1625, 22) (1625,)


In [16]:
#logistic regression
#random forest
#light gbm
#xgboost

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import numpy as np

pipe = Pipeline([("classifier",LogisticRegression())])

get_param = [
    {"classifier":[LogisticRegression()],
     "classifier__penalty":["l1","l2","elasticnet"],
     "classifier__C":np.logspace(0,4,10),
     #"classifier_fit_intercept" : ["True","False"],
     "classifier__solver" : ["newton-cg","lbfgs","liblinear"],
     "classifier__max_iter" : [100,150,200,300],
     #"classifier_class_weight": ["balanced",None]
     "classifier__verbose" : [2],
     "classifier__n_jobs" : [-1]},
    {
      "classifier":[RandomForestClassifier()],
      "classifier__n_estimators":[10,100,1000],
      "classifier__max_depth":[1,5,10],
      "classifier__max_features":["auto","sqrt","log2"],
      "classifier__min_samples_split":[2,5,10],
      "classifier__min_samples_leaf":[1,2,5],
      "classifier__bootstrap":[True,False]
    }
]


gridSearch = GridSearchCV(pipe,get_param,cv=10,verbose=1,n_jobs=-1,scoring="accuracy")
best_model = gridSearch.fit(x_train,y_train)

Fitting 10 folds for each of 846 candidates, totalling 8460 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
3620 fits failed out of a total of 8460.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
400 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 427, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/py

In [17]:
print(best_model.best_estimator_)

Pipeline(steps=[('classifier',
                 RandomForestClassifier(max_depth=10, n_estimators=10))])


In [18]:
print("The mean accuracy of the model is:",best_model.score(x_test,y_test))


The mean accuracy of the model is: 1.0


In [22]:
gridSearch

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('classifier', LogisticRegression())]),
             n_jobs=-1,
             param_grid=[{'classifier': [LogisticRegression()],
                          'classifier__C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04]),
                          'classifier__max_iter': [100, 150, 2...
                          'classifier__solver': ['newton-cg', 'lbfgs',
                                                 'liblinear'],
                          'classifier__verbose': [2]},
                         {'classifier': [RandomForestClassifier()],
                          'classifier__bootstrap': [True, False],
                          'classifier__max_depth': [1, 5, 10],
                          'classifier__max_features': ['auto', 'sqrt', 'log2'],
                          'classifier__min_samples_leaf': [1, 2, 5],
                          'classifier__min_samples_split': [2, 5, 10],
                          'classifier__n_estimators': [10, 100, 1000]}],
             verbose=1)

In [25]:
gridSearch.cv_results_

{'mean_fit_time': array([2.16562748e-03, 1.81069374e-03, 8.67334385e+00, 2.36173749e-01,
        2.68095326e-01, 1.93827057e-01, 1.99034214e-03, 2.63361931e-03,
        2.76212692e-03, 1.58374310e-03, 1.85594559e-03, 9.06272242e+00,
        1.78633356e-01, 2.70433760e-01, 1.44073486e-01, 1.64074898e-03,
        1.41236782e-03, 1.73051357e-03, 1.62096024e-03, 3.29775810e-03,
        8.52615359e+00, 2.69113660e-01, 4.84218597e-01, 1.50224614e-01,
        4.76715565e-03, 3.47735882e-03, 3.76615524e-03, 4.08084393e-03,
        2.82871723e-03, 8.84449809e+00, 1.55045485e-01, 6.20481730e-01,
        1.77392507e-01, 1.79901123e-03, 3.04486752e-03, 2.77574062e-03,
        1.38525963e-03, 4.29730415e-03, 1.02875977e+01, 2.43614888e-01,
        1.74202013e-01, 1.53060961e-01, 3.61285210e-03, 3.70190144e-03,
        2.54418850e-03, 2.30183601e-03, 3.30102444e-03, 1.11950186e+01,
        2.24417090e-01, 2.62004089e-01, 1.77012777e-01, 4.03347015e-03,
        3.13367844e-03, 3.73022556e-03, 3.03025

In [24]:
grid_df= pd.DataFrame(gridSearch.cv_results_)[['max_depth','n_estimators','mean_test_score']]
grid_df

KeyError: "['max_depth', 'n_estimators'] not in index"